In [1]:
# imports
import scipy.io
import numpy as np               # for arrays
from numpy import linalg as LA   # for eigenvalues
import matplotlib                # for plots
import time                      # for time measurements
from PIL import Image            # for showing images
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sympy

In [2]:
def show_img(img):
    temp = img.copy()
    temp.resize((46,56))
    im = Image.fromarray(temp.T)
    im.show()

Get training/test splits

In [3]:
# load data
mat = scipy.io.loadmat('face.mat')
raw_data = mat['X']

raw_data = np.transpose(raw_data)
N,D = raw_data.shape
C = 52 # number of classes in dataset
train_size = int(N * 0.8)
test_size = int(N * 0.2)

pca_training_data = np.empty([int(520*0.8), 2576])
pca_testing_data = np.empty([int(520*0.2), 2576])
lda_training_data = []
lda_testing_data = []

# create training and test data
for x in range(52):
    # 8/2 ratio for training and testing datasets
    lda_training_data.append(raw_data[x*10:x*10+8].copy())
    lda_testing_data.append(raw_data[x*10+8:(x+1)*10].copy())
    

lda_training_data = np.array(lda_training_data)
lda_testing_data = np.array(lda_testing_data)
pca_training_data = lda_training_data.reshape(train_size, D)
pca_testing_data = lda_testing_data.reshape(test_size,D)


In [4]:
print(pca_training_data.shape)

(416, 2576)


We need to make sure that the generalised eigenvalue problem that we encounter when doing LDA is solvable by making sure that the within-class scatter matrix is non-singular.
We do this by first reducing the dimension of the data via low dim PCA to an M <= N - c.

In [34]:
def get_Wpca(data, out_dim):
    # low-dim PCA
    S = data.dot(data.T)
    w, v = LA.eig(S)
    u = data.T.dot(v)
    u /= LA.norm(u,ord=2,axis=0)
    u = u.T
    id = np.argsort(np.abs(w))[::-1]
    w = w[id]
    u = u[id]

    return u[0:out_dim].T
    

We want a projection that maximises the ratio between the between-class scatter matrix and the within class scatter matrix.
The projection W turns out to be the solutions to the generalised eigen value problem. (Found via solving the langrangian. Slide 10-11)

In [6]:
def get_Wlda(Sb_data, Sw_data):
    #code that gets value
    lda_evals, lda_evecs = LA.eig(LA.inv(Sw_data).dot(Sb_data))
    return lda_evecs

In [15]:
mean_all_data = pca_training_data.T.mean(axis=1).T

# between class scatter (scalar)
mean_class_data = lda_training_data.mean(axis=1)
diff_class_mean = mean_class_data - mean_all_data
print(mean_class_data.shape)

Sb = np.dot(diff_class_mean.T, diff_class_mean)

# within class scatter (scalar)
diff_class_data = lda_training_data - mean_class_data.reshape(52,1,-1)


(52, 2576)


In [16]:
print(diff_class_data.shape)


(52, 8, 2576)


In [31]:
Sw = np.zeros((2576, 2576));
for x in diff_class_data:
    Sw += np.dot(x.T,x)

St = Sb + Sw
Wpca = get_Wpca(pca_training_data, -1)
reduced_Sb = Wpca.T.dot(Sb).dot(Wpca)
reduced_Sw = Wpca.T.dot(Sw).dot(Wpca)
Wlda = get_Wlda(reduced_Sb, reduced_Sw)

Wopt = Wlda.T.dot(Wpca.T)

(416, 2576)
1.0000000000000002
416


In [35]:
print(reduced_Sw)


[[ 1.67682201e+08 -2.31842406e+07  7.61048119e+06 ... -7.88422965e+07
  -4.80474464e+07 -9.39422913e+07]
 [-2.31842406e+07  9.37111266e+07 -4.05044952e+07 ...  4.22688546e+07
  -1.50252789e+07 -1.27530266e+07]
 [ 7.61048119e+06 -4.05044952e+07  4.33511832e+07 ... -3.12789450e+07
   1.28109770e+07  6.30083241e+06]
 ...
 [-7.88422965e+07  4.22688546e+07 -3.12789450e+07 ...  8.59414168e+07
   9.08971028e+06  4.08201001e+07]
 [-4.80474464e+07 -1.50252789e+07  1.28109770e+07 ...  9.08971028e+06
   3.53326375e+07  3.39553206e+07]
 [-9.39422913e+07 -1.27530266e+07  6.30083241e+06 ...  4.08201001e+07
   3.39553206e+07  7.06597261e+07]]


In [ ]:
# quick test to see if training data can be reconstructed correctly
training_weights = pca_training_data.dot(Wopt.T)

We can now express each data point as product of a weight vector with Wopt. This weight vector can be used to classify each data point using nearest neighbour

In [ ]:
print(training_weights.shape)

In [ ]:
correct = True
for idx,x in enumerate(training_weights):
    reconstructed_face = Wopt.T.dot(x)
    show_img(reconstructed_face)
    